In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os 
from warnings import simplefilter
import pandas as pd
from imblearn.over_sampling import SMOTE  


import models
import class_sampling
import train
import metric_utils
import inference
import loss_fns
import torchvision.ops 

NUM_CLASSES = 10
n_epochs = 30
batch_size_train = 64
batch_size_test = 1000
momentum = 0

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

CLASS_LABELS = {'airplane': 0,
                 'automobile': 1,
                 'bird': 2,
                 'cat': 3,
                 'deer': 4,
                 'dog': 5,
                 'frog': 6,
                 'horse': 7,
                 'ship': 8,
                 'truck': 9}


simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=DeprecationWarning)

col_names = ["name", 
            "num_classes", 
            "classes_used", 
            "ratio", 
            "learning_rate", 
            "mean_0", "variance_0",
            "mean_10", "variance_10",
            "mean_20", "variance_20",
            "mean_30", "variance_30",
          #   "mean_40", "variance_40",
          #   "mean_50", "variance_50",
             "cap", "normalization"]

rows = []

In [2]:
# 3 classes

NUM_CLASSES_REDUCED = 3
nums = (0, 3, 1)
ratio = (20, 2, 1)

norm=True

if norm:
    transform=torchvision.transforms.Compose([torchvision.transforms.Normalize(mean=[134.1855, 122.7346, 118.3749], std=[70.5125, 64.4848, 66.5604])])
else:
    transform=None

    
train_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=True, download=True,
                             transform=transform)  

test_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=False, download=True,
                             transform=transform)  

train_CIFAR10.data = train_CIFAR10.data.reshape(50000, 3, 32, 32)
test_CIFAR10.data = test_CIFAR10.data.reshape(10000, 3, 32, 32)


reduced_train_CIFAR10 = class_sampling.Reduce(train_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)
reduced_test_CIFAR10 = class_sampling.Reduce(test_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)

ratio_train_CIFAR10 = class_sampling.Ratio(train_CIFAR10, NUM_CLASSES_REDUCED, ratio, nums=nums, transform=transform)
targets = ratio_train_CIFAR10.labels 
class_count = np.unique(targets, return_counts=True)[1]

smote_train_CIFAR10 = class_sampling.Smote(ratio_train_CIFAR10, 5000 * NUM_CLASSES_REDUCED, transform=transform)

triplet_train_CIFAR10 = class_sampling.ForTripletLoss(reduced_train_CIFAR10, smote=False, transform=transform, num_classes=3)
triplet_ratio_train_CIFAR10 = class_sampling.ForTripletLoss(ratio_train_CIFAR10, smote=False, transform=transform, num_classes=3)
triplet_smote_train_CIFAR10 = class_sampling.ForTripletLoss(smote_train_CIFAR10, smote=True, transform=transform, num_classes=3)

weight = 1. / class_count
samples_weight = weight[targets]
samples_weight = torch.from_numpy(samples_weight)
oversampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(max(class_count) * NUM_CLASSES_REDUCED), replacement=True)
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
undersampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * NUM_CLASSES_REDUCED), replacement=False)

weight *= max(class_count)

train_loader_reduced = DataLoader(reduced_train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_ratio = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True) 

train_loader_oversampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=oversampler)

train_loader_undersampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler)

train_loader_sampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=sampler)

train_loader_smote = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss = DataLoader(triplet_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_ratio = DataLoader(triplet_ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_smote = DataLoader(triplet_smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

test_loader_reduced = DataLoader(reduced_test_CIFAR10, batch_size=batch_size_test, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# 3 class normal

learning_rates = [1e-4, 1e-3, 5e-4]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_reduced, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["normal", 3, nums, (1, 1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)

In [ ]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [8]:
#  3 class ratio

learning_rates =  [1e-4, 1e-3, 5e-4]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_ratio, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["ratio", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)


Test set: Avg. loss: 0.0011061461766560872, AUC: 0.42147850000000003


Test set: Avg. loss: 0.0011229241689046225, AUC: 0.43375308333333323


Test set: Avg. loss: 0.0011965232292811077, AUC: 0.50397825


Test set: Avg. loss: 0.0013694471518198648, AUC: 0.5547101666666666


Test set: Avg. loss: 0.0010940184195836384, AUC: 0.5872949166666667


Test set: Avg. loss: 0.0011234662532806396, AUC: 0.6271675


Test set: Avg. loss: 0.0012737938165664673, AUC: 0.629696


Test set: Avg. loss: 0.0014157256285349529, AUC: 0.6302791666666666


Test set: Avg. loss: 0.0011149876117706298, AUC: 0.5022431666666667


Test set: Avg. loss: 0.00111220649878184, AUC: 0.5008525833333334


Test set: Avg. loss: 0.0012121287981669109, AUC: 0.56545225


Test set: Avg. loss: 0.0014367966651916504, AUC: 0.5884317499999999


Test set: Avg. loss: 0.0011054763793945312, AUC: 0.4172478333333333


Test set: Avg. loss: 0.001166960875193278, AUC: 0.43572633333333327


Test set: Avg. loss: 0.0013708451986312866, AUC: 0.493

In [9]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [10]:
# 3 class oversampled 

learning_rates = [1e-3, 5e-4, 1e-4]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_oversampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["oversampled", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)


Test set: Avg. loss: 0.0010947994788487753, AUC: 0.56657275


Test set: Avg. loss: 0.0010222713152567545, AUC: 0.7476595000000001


Test set: Avg. loss: 0.0009208483298619589, AUC: 0.8058216666666667


Test set: Avg. loss: 0.0008615500529607137, AUC: 0.8321021666666667


Test set: Avg. loss: 0.0011039157311121624, AUC: 0.42707975000000004


Test set: Avg. loss: 0.0010247631072998046, AUC: 0.7248186666666667


Test set: Avg. loss: 0.0009506714145342509, AUC: 0.785685


Test set: Avg. loss: 0.0008906665245691936, AUC: 0.8202210000000001


Test set: Avg. loss: 0.0011069478988647461, AUC: 0.5771989166666667


Test set: Avg. loss: 0.0009920546213785808, AUC: 0.7478355833333333


Test set: Avg. loss: 0.0009075014392534892, AUC: 0.79576525


Test set: Avg. loss: 0.0008648690780003865, AUC: 0.81608775


Test set: Avg. loss: 0.0011047943433125814, AUC: 0.462027


Test set: Avg. loss: 0.0010138102769851685, AUC: 0.7138214166666668


Test set: Avg. loss: 0.0009392054279645284, AUC: 0.78475783333

In [11]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [12]:
#  3 class SMOTE

learning_rates = [1e-4, 5e-4, 1e-3]

learning_rate_aucs = []

loss_fn_args = {}
loss_fn_args['loss_cap'] = None

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax_with_smote(epoch, train_loader_smote, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["smote", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)


Test set: Avg. loss: 0.0011024348735809326, AUC: 0.5284704166666667


Test set: Avg. loss: 0.0010927425622940063, AUC: 0.6219938333333334


Test set: Avg. loss: 0.0010809800624847412, AUC: 0.68663525


Test set: Avg. loss: 0.001063308556874593, AUC: 0.7213240833333333


Test set: Avg. loss: 0.001103477398554484, AUC: 0.47049291666666665


Test set: Avg. loss: 0.0010985154310862223, AUC: 0.5479706666666667


Test set: Avg. loss: 0.001090846339861552, AUC: 0.6222283333333333


Test set: Avg. loss: 0.0010767723321914672, AUC: 0.6630774166666668


Test set: Avg. loss: 0.0010997201601664224, AUC: 0.5258056666666667


Test set: Avg. loss: 0.0010976532300313313, AUC: 0.55747225


Test set: Avg. loss: 0.0010960997740427653, AUC: 0.5830667500000001


Test set: Avg. loss: 0.0010943124294281006, AUC: 0.6087030833333333


Test set: Avg. loss: 0.001101622740427653, AUC: 0.4751273333333333


Test set: Avg. loss: 0.0010785504976908366, AUC: 0.6937432499999999


Test set: Avg. loss: 0.001063970088958

In [13]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [14]:
# 3 class capped smote 


momentum=0
learning_rates = [1e-3, 5e-4, 1e-4]


cap_aucs = []

caps = [1, 5, 10]

for cap in caps:
    
    loss_fn_args = {}
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNet(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                _, _ = train.train_softmax_with_smote(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = caps[c]
    for i in range(len(learning_rates)): 
        row = ["capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)


Test set: Avg. loss: 0.0011089017788569132, AUC: 0.5046929166666666


Test set: Avg. loss: 0.001574310541152954, AUC: 0.6560488333333333


Test set: Avg. loss: 0.0015633835792541503, AUC: 0.685893


Test set: Avg. loss: 0.0015301477511723836, AUC: 0.6995961666666667


Test set: Avg. loss: 0.0011047988732655843, AUC: 0.4485218333333334


Test set: Avg. loss: 0.0016434243122736612, AUC: 0.6248416666666667


Test set: Avg. loss: 0.001616351842880249, AUC: 0.6463213333333334


Test set: Avg. loss: 0.001616426666577657, AUC: 0.6586984166666667


Test set: Avg. loss: 0.0010929834445317587, AUC: 0.59649675


Test set: Avg. loss: 0.0016810382604599, AUC: 0.6184855833333333


Test set: Avg. loss: 0.0016556649605433146, AUC: 0.6387421666666667


Test set: Avg. loss: 0.0016343777974446615, AUC: 0.6486046666666666


Test set: Avg. loss: 0.0011026423374811808, AUC: 0.52387


Test set: Avg. loss: 0.001606564203898112, AUC: 0.6057905


Test set: Avg. loss: 0.0016171462933222453, AUC: 0.6415721666666


Test set: Avg. loss: 0.001016865611076355, AUC: 0.7214776666666666


Test set: Avg. loss: 0.0009618866840998332, AUC: 0.7313865


Test set: Avg. loss: 0.0009408379594484965, AUC: 0.7478806666666666


Test set: Avg. loss: 0.0011057717800140381, AUC: 0.44706758333333335


Test set: Avg. loss: 0.0010132566690444946, AUC: 0.6974945833333334


Test set: Avg. loss: 0.0009697758158047995, AUC: 0.7195119999999999


Test set: Avg. loss: 0.0009472548166910808, AUC: 0.7383986666666668


Test set: Avg. loss: 0.001104608138402303, AUC: 0.47800099999999995


Test set: Avg. loss: 0.001021358609199524, AUC: 0.67475775


Test set: Avg. loss: 0.0009923197627067567, AUC: 0.6997320833333335


Test set: Avg. loss: 0.0009891886909802754, AUC: 0.71357275


Test set: Avg. loss: 0.00110628612836202, AUC: 0.40635058333333335


Test set: Avg. loss: 0.0010014968315760295, AUC: 0.6926616666666666


Test set: Avg. loss: 0.0009727685252825419, AUC: 0.7189756666666666


Test set: Avg. loss: 0.0009684410691261292, AU


Test set: Avg. loss: 0.0009719972014427185, AUC: 0.7290505


Test set: Avg. loss: 0.0009754973649978637, AUC: 0.7297843333333334


Test set: Avg. loss: 0.0011115421454111734, AUC: 0.39468024999999995


Test set: Avg. loss: 0.0010144439140955606, AUC: 0.7207508333333333


Test set: Avg. loss: 0.0009580605427424113, AUC: 0.7364392500000001


Test set: Avg. loss: 0.0009509671926498413, AUC: 0.7410335


Test set: Avg. loss: 0.001096882939338684, AUC: 0.554111


Test set: Avg. loss: 0.001015723983446757, AUC: 0.6777006666666666


Test set: Avg. loss: 0.0009884311159451802, AUC: 0.6946431666666667


Test set: Avg. loss: 0.0009627226789792379, AUC: 0.7193


Test set: Avg. loss: 0.0010926958322525025, AUC: 0.58256075


Test set: Avg. loss: 0.0009930734435717266, AUC: 0.6946941666666667


Test set: Avg. loss: 0.0009740681250890096, AUC: 0.7106511666666666


Test set: Avg. loss: 0.0009724552830060323, AUC: 0.7214235000000001


Test set: Avg. loss: 0.0011114898522694907, AUC: 0.4406507499999999


In [15]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [16]:
# 3 class euclidean distance capped smote 


momentum=0
learning_rates = [1e-3, 5e-4, 1e-4]


cap_aucs = []

caps = [1, 5, 10]

loss_fn_args = {}
loss_fn_args['distance'] = 'euclidean'


for cap in caps:
    
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNetWithEmbeddings(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                _, _ = train.train_softmax_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = caps[c]
    for i in range(len(learning_rates)): 
        row = ["distance_capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)



Test set: Avg. loss: 0.0010936386982599894, AUC: 0.6369764166666666


Test set: Avg. loss: 0.0011055919726689657, AUC: 0.6861483333333332


Test set: Avg. loss: 0.0011112023194630942, AUC: 0.7031156666666666


Test set: Avg. loss: 0.0010712438821792603, AUC: 0.7245791666666666


Test set: Avg. loss: 0.001098901589711507, AUC: 0.5035550833333333


Test set: Avg. loss: 0.0011472790241241456, AUC: 0.6577565


Test set: Avg. loss: 0.0011363714933395387, AUC: 0.679912


Test set: Avg. loss: 0.0011391181945800781, AUC: 0.7001859166666667


Test set: Avg. loss: 0.0011044926643371583, AUC: 0.47181349999999994


Test set: Avg. loss: 0.0011109636227289836, AUC: 0.6991806666666668


Test set: Avg. loss: 0.0010984382629394532, AUC: 0.7210651666666665


Test set: Avg. loss: 0.0010675038894017537, AUC: 0.7430861666666667


Test set: Avg. loss: 0.0011056922674179076, AUC: 0.492054


Test set: Avg. loss: 0.0011312808195749918, AUC: 0.68086425


Test set: Avg. loss: 0.0011157657305399576, AUC: 0.70606


Test set: Avg. loss: 0.0010006168087323506, AUC: 0.7007906666666667


Test set: Avg. loss: 0.000991052786509196, AUC: 0.7465673333333335


Test set: Avg. loss: 0.000992796262105306, AUC: 0.7577124999999999


Test set: Avg. loss: 0.0011017233928044636, AUC: 0.49381433333333336


Test set: Avg. loss: 0.001044129490852356, AUC: 0.676765


Test set: Avg. loss: 0.0010413487354914348, AUC: 0.7089582499999999


Test set: Avg. loss: 0.001047049840291341, AUC: 0.7347656666666667


Test set: Avg. loss: 0.0011043426195780435, AUC: 0.41822224999999996


Test set: Avg. loss: 0.0010062861442565918, AUC: 0.6886078333333333


Test set: Avg. loss: 0.0009938278992970785, AUC: 0.7191567499999999


Test set: Avg. loss: 0.0009720776081085205, AUC: 0.7530308333333333


Test set: Avg. loss: 0.0010985084772109986, AUC: 0.5633248333333333


Test set: Avg. loss: 0.0010073503057161967, AUC: 0.7039656666666666


Test set: Avg. loss: 0.0010310287872950237, AUC: 0.7285898333333334


Test set: Avg. loss: 0.00100426


Test set: Avg. loss: 0.0010055485367774964, AUC: 0.7125833333333333


Test set: Avg. loss: 0.0009834045171737671, AUC: 0.7223613333333333


Test set: Avg. loss: 0.001110142191251119, AUC: 0.3632611666666667


Test set: Avg. loss: 0.0009840183854103088, AUC: 0.7084878333333333


Test set: Avg. loss: 0.0009811877806981404, AUC: 0.7262844999999999


Test set: Avg. loss: 0.0009945996999740601, AUC: 0.7435213333333333


Test set: Avg. loss: 0.001106520414352417, AUC: 0.4813669166666667


Test set: Avg. loss: 0.0010102336009343465, AUC: 0.7366194999999999


Test set: Avg. loss: 0.000978839119275411, AUC: 0.7450051666666666


Test set: Avg. loss: 0.0009691664377848308, AUC: 0.7418221666666667


Test set: Avg. loss: 0.0011050666173299153, AUC: 0.3887366666666667


Test set: Avg. loss: 0.0009948948224385579, AUC: 0.689836


Test set: Avg. loss: 0.0009786287546157838, AUC: 0.7108009166666666


Test set: Avg. loss: 0.0009680883487065633, AUC: 0.7248308333333333


Test set: Avg. loss: 0.001108383

In [17]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [18]:
# 3 class cosine distance capped smote 


momentum=0
learning_rates = [1e-3, 5e-4, 1e-4]


cap_aucs = []

caps = [1, 5, 10]

loss_fn_args = {}
loss_fn_args['distance'] = 'cosine'


for cap in caps:
    
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNetWithEmbeddings(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                _, _ = train.train_softmax_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = caps[c]
    for i in range(len(learning_rates)): 
        row = ["cosine_distance_capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)



Test set: Avg. loss: 0.0011055415074030558, AUC: 0.38005533333333336


Test set: Avg. loss: 0.0010076440175374349, AUC: 0.7001938333333334


Test set: Avg. loss: 0.0009683122436205546, AUC: 0.712987


Test set: Avg. loss: 0.0009642910559972128, AUC: 0.7338803333333335


Test set: Avg. loss: 0.0010966946681340536, AUC: 0.5500838333333333


Test set: Avg. loss: 0.0009827250440915425, AUC: 0.7104226666666666


Test set: Avg. loss: 0.0009693577686945597, AUC: 0.7252480833333333


Test set: Avg. loss: 0.0009774929285049439, AUC: 0.7279855


Test set: Avg. loss: 0.001093033711115519, AUC: 0.57653375


Test set: Avg. loss: 0.0009999072551727295, AUC: 0.6984451666666667


Test set: Avg. loss: 0.0009814204374949138, AUC: 0.7221749166666668


Test set: Avg. loss: 0.0009723171393076579, AUC: 0.7307384166666667


Test set: Avg. loss: 0.0010961555242538451, AUC: 0.5658584999999999


Test set: Avg. loss: 0.000990861475467682, AUC: 0.7180896666666666


Test set: Avg. loss: 0.0009716771443684896, AUC


Test set: Avg. loss: 0.0009933680097262065, AUC: 0.6939047500000001


Test set: Avg. loss: 0.0009753385782241821, AUC: 0.7173980000000001


Test set: Avg. loss: 0.00098874165614446, AUC: 0.7309538333333333


Test set: Avg. loss: 0.0011000401576360067, AUC: 0.5157429166666666


Test set: Avg. loss: 0.0009977991382280986, AUC: 0.6880851666666666


Test set: Avg. loss: 0.0009763535857200623, AUC: 0.7121859166666665


Test set: Avg. loss: 0.0009658528765042623, AUC: 0.7250770000000001


Test set: Avg. loss: 0.001096102754275004, AUC: 0.5486070833333333


Test set: Avg. loss: 0.0010320355892181397, AUC: 0.6862285833333331


Test set: Avg. loss: 0.0009796069463094076, AUC: 0.7096381666666667


Test set: Avg. loss: 0.0009827236930529276, AUC: 0.7238424166666665


Test set: Avg. loss: 0.0010975841681162516, AUC: 0.5274588333333333


Test set: Avg. loss: 0.0009785786271095276, AUC: 0.7142916666666667


Test set: Avg. loss: 0.0009575179815292359, AUC: 0.7239957499999999


Test set: Avg. loss: 0


Test set: Avg. loss: 0.0009509822328885397, AUC: 0.7341509999999999


Test set: Avg. loss: 0.0009344067573547364, AUC: 0.7491564999999999


Test set: Avg. loss: 0.00109254789352417, AUC: 0.6488634166666667


Test set: Avg. loss: 0.0009836358030637104, AUC: 0.7124772500000001


Test set: Avg. loss: 0.0009623388648033142, AUC: 0.7287971666666667


Test set: Avg. loss: 0.0009584565758705139, AUC: 0.7339683333333333


Test set: Avg. loss: 0.0011012711922327677, AUC: 0.5230758333333333


Test set: Avg. loss: 0.0009854709307352701, AUC: 0.70752


Test set: Avg. loss: 0.0009705777764320374, AUC: 0.7231626666666666


Test set: Avg. loss: 0.001023399511973063, AUC: 0.7213120000000001


Test set: Avg. loss: 0.0010993149677912394, AUC: 0.5271454166666667


Test set: Avg. loss: 0.0009839840531349182, AUC: 0.7254368333333335


Test set: Avg. loss: 0.0009517814119656881, AUC: 0.7478110833333332


Test set: Avg. loss: 0.0009533281127611797, AUC: 0.7558744166666665


Test set: Avg. loss: 0.0011024358

In [19]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [ ]:
# 3 class triplet loss capped smote


momentum=0
learning_rates = [1e-3, 5e-4, 1e-4]

cap_aucs = []

start_epoch = 2

loss_caps = [1, 5, 10]
loss_fn_args = {}


for loss_cap in loss_caps:
    
    loss_fn_args['loss_cap'] = loss_cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNetWithEmbeddings(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                loss_fn_args['loss_cap'] = None
                _, _ = train.train_softmax_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
            for epoch in range(start_epoch, n_epochs + 1):
                loss_fn_args['loss_cap'] = loss_cap
                _, _ = train.train_triplet_capped_loss(epoch, train_loader_tripletloss_smote, network, optimizer, verbose=False, cap_calc=loss_fns.TripletLoss,loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = caps[c]
    for i in range(len(learning_rates)): 
        row = ["triplet_loss_capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)



Test set: Avg. loss: 0.001090798298517863, AUC: 0.6275455833333333


Test set: Avg. loss: 0.0010112351179122924, AUC: 0.6849049166666666


Test set: Avg. loss: 0.0009967645009358725, AUC: 0.7066909999999998


Test set: Avg. loss: 0.0009790867567062377, AUC: 0.7257449166666667


Test set: Avg. loss: 0.0010908244848251343, AUC: 0.6068476666666668


Test set: Avg. loss: 0.001025878111521403, AUC: 0.69062775


Test set: Avg. loss: 0.001001077930132548, AUC: 0.7137581666666666


Test set: Avg. loss: 0.0010044774611790974, AUC: 0.7283343333333333


Test set: Avg. loss: 0.0010957770744959513, AUC: 0.5945498333333333


Test set: Avg. loss: 0.0010208751757939656, AUC: 0.6770616666666666


Test set: Avg. loss: 0.001009067436059316, AUC: 0.7041759166666667


Test set: Avg. loss: 0.0010336569150288899, AUC: 0.7188218333333333


Test set: Avg. loss: 0.0010945513248443604, AUC: 0.5797924166666667


Test set: Avg. loss: 0.0010250928401947022, AUC: 0.6944658333333334


Test set: Avg. loss: 0.00103488


Test set: Avg. loss: 0.0011042052110036215, AUC: 0.47445875


Test set: Avg. loss: 0.0010264458656311034, AUC: 0.6871260833333332


Test set: Avg. loss: 0.00100686909755071, AUC: 0.7113415000000001


Test set: Avg. loss: 0.0009901551008224486, AUC: 0.727234


Test set: Avg. loss: 0.0011103670994440714, AUC: 0.3206058333333333


Test set: Avg. loss: 0.0010133507649103801, AUC: 0.681817


Test set: Avg. loss: 0.0010078470309575398, AUC: 0.7050957499999999


Test set: Avg. loss: 0.0010084237257639568, AUC: 0.7230279999999999


Test set: Avg. loss: 0.0011009645064671834, AUC: 0.48435758333333334


Test set: Avg. loss: 0.0010037092765172323, AUC: 0.6946527499999999


Test set: Avg. loss: 0.000972671627998352, AUC: 0.7302672500000001


Test set: Avg. loss: 0.0009687488079071045, AUC: 0.742565


Test set: Avg. loss: 0.0011146032412846882, AUC: 0.39922541666666667


Test set: Avg. loss: 0.0009936522046724956, AUC: 0.6980645


Test set: Avg. loss: 0.0009849374294281007, AUC: 0.72684025


Test 

In [ ]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 